# This notebook shows example to load the CHGNet for prediction


In [ ]:
try:
    from chgnet.model import CHGNet
except ImportError:
    # install CHGNet (only needed on Google Colab or if you didn't install CHGNet yet)
    !pip install chgnet

In [ ]:
import numpy as np
from pymatgen.core import Structure

from chgnet.model import CHGNet

# If the above line fails in Google Colab due to numpy version issue,
# please restart the runtime, and the problem will be solved

np.set_printoptions(precision=4, suppress=True)

### Read structure from a json or cif file


In [ ]:
try:
    from chgnet import ROOT

    structure = Structure.from_file(f"{ROOT}/examples/mp-18767-LiMnO2.cif")
except Exception:
    from urllib.request import urlopen

    url = "https://raw.githubusercontent.com/CederGroupHub/chgnet/main/examples/mp-18767-LiMnO2.cif"
    cif = urlopen(url).read().decode("utf-8")
    structure = Structure.from_str(cif, fmt="cif")

print(structure)

Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   2.868779   4.634475   5.832507
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Li+   0.5  0.5  0.37975
  1  Li+   0    0    0.62025
  2  Mn3+  0.5  0.5  0.863252
  3  Mn3+  0    0    0.136747
  4  O2-   0.5  0    0.360824
  5  O2-   0    0.5  0.098514
  6  O2-   0.5  0    0.901486
  7  O2-   0    0.5  0.639176


### Define Model


In [ ]:
chgnet = CHGNet.load()

# Alternatively you can read your own model
# chgnet = CHGNet.from_file(model_path)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu


# Predict energy, force, stress, magmom


In [ ]:
prediction = chgnet.predict_structure(structure)

for key, unit in [
    ("energy", "eV/atom"),
    ("forces", "eV/A"),
    ("stress", "GPa"),
    ("magmom", "mu_B"),
]:
    print(f"CHGNet-predicted {key} ({unit}):\n{prediction[key[0]]}\n")

CHGNet-predicted energy (eV/atom):
-7.367691516876221

CHGNet-predicted forces (eV/A):
[[ 0.     -0.      0.0238]
 [ 0.      0.     -0.0238]
 [ 0.     -0.      0.0926]
 [-0.     -0.     -0.0926]
 [ 0.      0.     -0.0024]
 [-0.     -0.     -0.0131]
 [ 0.      0.      0.0131]
 [-0.      0.      0.0024]]

CHGNet-predicted stress (GPa):
[[-0.3037 -0.      0.    ]
 [-0.      0.2231 -0.    ]
 [ 0.     -0.     -0.1074]]

CHGNet-predicted magmom (mu_B):
[0.003  0.003  3.8694 3.8694 0.0441 0.0386 0.0386 0.0441]



# Structure Optimization


In [ ]:
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu


In [ ]:
# Perturb the structure
structure.perturb(0.1)

# Relax the perturbed structure
result = relaxer.relax(structure, verbose=True)
print("Relaxed structure:\n")
print(result["final_structure"])

      Step     Time          Energy          fmax
FIRE:    0 14:37:08      -58.683239        2.601857
FIRE:    1 14:37:08      -58.812550        1.642414
FIRE:    2 14:37:08      -58.905983        0.343259
FIRE:    3 14:37:08      -58.881161        1.028341
FIRE:    4 14:37:09      -58.887203        0.944485
FIRE:    5 14:37:09      -58.897339        0.789977
FIRE:    6 14:37:09      -58.908859        0.598800
FIRE:    7 14:37:09      -58.918888        0.364584
FIRE:    8 14:37:09      -58.924923        0.167921
FIRE:    9 14:37:10      -58.926193        0.189265
FIRE:   10 14:37:10      -58.926289        0.186574
FIRE:   11 14:37:10      -58.926487        0.181227
FIRE:   12 14:37:10      -58.926773        0.176636
FIRE:   13 14:37:10      -58.927135        0.171366
FIRE:   14 14:37:10      -58.927570        0.165129
FIRE:   15 14:37:11      -58.928059        0.158151
FIRE:   16 14:37:11      -58.928581        0.150665
FIRE:   17 14:37:11      -58.929195        0.142299
FIRE:   18 14:

# Molecular Dynamics


In [ ]:
from chgnet.model.dynamics import MolecularDynamics

md = MolecularDynamics(
    atoms=structure,
    model=chgnet,
    ensemble="nvt",
    temperature=1000,  # in k
    timestep=2,  # in fs
    trajectory="md_out.traj",
    logfile="md_out.log",
    loginterval=100,
)
md.run(50)  # run a 0.1 ps MD simulation

CHGNet will run on cpu


/Users/janosh/.venv/py310/lib/python3.10/site-packages/ase/md/nvtberendsen.py:95: RuntimeWarning: divide by zero encountered in scalar divide
  (self.temperature / old_temperature - 1.0) *


# Magmom Visualization


In [ ]:
supercell = structure.make_supercell([2, 2, 2], in_place=False)
print(supercell.composition)

Li+16 Mn3+16 O2-32


In [ ]:
import random

n_Li = int(supercell.composition["Li+"])
remove_ids = random.sample(list(range(n_Li)), n_Li // 2)

supercell.remove_sites(remove_ids)
print(supercell.composition)

Li+8 Mn3+16 O2-32


In [ ]:
result = relaxer.relax(supercell)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 19:33:07     -426.191837*      20.8815
FIRE:    1 19:33:07     -400.066006*     268.6776
FIRE:    2 19:33:07     -426.080700*      50.7814
FIRE:    3 19:33:08     -427.411331*       4.2803
FIRE:    4 19:33:08     -426.366394*      52.4735
FIRE:    5 19:33:08     -426.988091*      35.4947
FIRE:    6 19:33:09     -427.518917*       9.0208
FIRE:    7 19:33:09     -427.456432*      15.6711
FIRE:    8 19:33:09     -427.474777*      14.7084
FIRE:    9 19:33:09     -427.507435*      12.8331
FIRE:   10 19:33:10     -427.547436*      10.1189
FIRE:   11 19:33:10     -427.586182*       6.6958
FIRE:   12 19:33:11     -427.616276*       2.8986
FIRE:   13 19:33:11     -427.633926*       1.8915
FIRE:   14 19:33:12     -427.641483*       4.3156
FIRE:   15 19:33:12     -427.646664*       7.0416
FIRE:   16 19:33:13     -427.658947*       8.6467
FIRE:   17 19:33:14     -427.687492*       8.7184
FI

In [ ]:
import pandas as pd

df_magmom = pd.DataFrame({"Unrelaxed": chgnet.predict_structure(supercell)["m"]})
df_magmom["CHGNet relaxed"] = result["final_structure"].site_properties["magmom"]

In [ ]:
fig = df_magmom.hist(
    nbins=200,
    sharex=True,
    sharey=True,
    backend="plotly",
    barmode="overlay",
    layout={"title": "Magmom distribution"},
    opacity=0.7,
    range_x=[3, 4],
    template="plotly_white",
)
fig.layout.legend.update(title="", x=1, y=1, xanchor="right", yanchor="top")
fig.layout.xaxis.title = "Magnetic moment"
fig.show()